<div align="center">
  <img src="logo_branding.png" width="250" alt="kavi.ai Logo">
  <h1>DoRA: Weight-Decomposed Precision</h1>
  <p><b>A Premium Training Module by kavi.ai</b></p>
</div>

---

### 💎 **Smarter Overview**
Weight-Decomposed Low-Rank Adaptation (DoRA) solves the gradient mismatch in standard LoRA by separately optimizing magnitude and direction.

### 🚀 **Enterprise Use Case**
High-fidelity training for scientific research or mathematical reasoning where standard LoRA fails to capture the intricate nuances of the task.

### 📈 **Strategic Advantages**
- **Better Convergence**: Faster training cycles with more stable loss curves.
- **Parity with Full Tuning**: Achieves results that were previously only possible with expensive full-weight updates.
- **Architectural Awareness**: Adapts to the model's internal weight geometry more naturally.

---

### 🛠️ **Visualization of the Process**
<div align="center"><svg width="800" height="300" viewBox="0 0 800 300" xmlns="http://www.w3.org/2000/svg">
  <defs>
    <filter id="glow-blue"><feGaussianBlur stdDeviation="3" result="blur"/><feMerge><feMergeNode in="blur"/><feMergeNode in="SourceGraphic"/></feMerge></filter>
    <filter id="glow-orange"><feGaussianBlur stdDeviation="3" result="blur"/><feMerge><feMergeNode in="blur"/><feMergeNode in="SourceGraphic"/></feMerge></filter>
  </defs>
  <circle cx="50" cy="150" r="30" fill="#334155" stroke="#94a3b8" stroke-width="2" />
  <text x="50" y="155" font-family="Arial" font-size="12" fill="white" text-anchor="middle">Input X</text>
  <path d="M 80 150 L 150 150" stroke="#94a3b8" stroke-width="2" fill="none" />
  <rect x="200" y="40" width="200" height="60" rx="10" fill="#1e293b" stroke="#ef4444" stroke-width="2" />
  <text x="300" y="75" font-family="Arial" font-size="14" fill="#f87171" text-anchor="middle">Frozen Weights (W)</text>
  <rect x="200" y="200" width="80" height="60" rx="10" fill="#1e3a8a" stroke="#3b82f6" stroke-width="2" filter="url(#glow-blue)" />
  <text x="240" y="235" font-family="Arial" font-size="14" fill="white" text-anchor="middle">Matrix A</text>
  <path d="M 280 230 L 320 230" stroke="#3b82f6" stroke-width="2" fill="none" />
  <rect x="320" y="200" width="80" height="60" rx="10" fill="#1e3a8a" stroke="#3b82f6" stroke-width="2" filter="url(#glow-blue)" />
  <text x="360" y="235" font-family="Arial" font-size="14" fill="white" text-anchor="middle">Matrix B</text>
  <path d="M 150 150 L 150 70 L 200 70" stroke="#94a3b8" stroke-width="2" fill="none" />
  <path d="M 150 150 L 150 230 L 200 230" stroke="#94a3b8" stroke-width="2" fill="none" />
  <circle cx="550" cy="150" r="25" fill="#1e293b" stroke="#fbbf24" stroke-width="2" filter="url(#glow-orange)" />
  <text x="550" y="156" font-family="Arial" font-size="20" fill="#fbbf24" text-anchor="middle">+</text>
  <path d="M 400 70 L 525 150" stroke="#94a3b8" stroke-width="2" fill="none" />
  <path d="M 400 230 L 525 150" stroke="#3b82f6" stroke-width="2" fill="none" />
  <path d="M 575 150 L 650 150" stroke="#fbbf24" stroke-width="2" fill="none" />
  <text x="700" y="155" font-family="Arial" font-size="16" fill="white" text-anchor="middle">Output Y</text>
</svg></div>

---

In [ ]:
!pip install transformers --upgrade
!pip install datasets
!pip install trl[peft] --upgrade
!pip install -U git+https://github.com/huggingface/trl
!pip install bitsandbytes loralib
!pip install wandb -U
!pip install hf_transfer
!pip install sentencepiece


In [ ]:
%env HF_HUB_ENABLE_HF_TRANSFER=True
%env WANDB_PROJECT=LLM-Training-Course
%env WANDB_RUN_ID=DORA
%env WANDB_NOTEBOOK_NAME={__vsc_ipynb_file__}

In [ ]:
import wandb
wandb.login()

In [ ]:
import sys
sys.path.append('/root/llm-training-course/')

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

In [ ]:
from datasets import load_dataset
train_ds, eval_ds = load_dataset("mlabonne/orpo-dpo-mix-40k", split=["train[:10%]","train[10%:15%]"])

In [ ]:
train_ds

In [ ]:
train_ds = train_ds.map(lambda x: { "messages": [{"role":"user", "content": x["prompt"] + "\n" +  x["chosen"][0]["content"]}] + x["chosen"][1:] })
eval_ds = eval_ds.map(lambda x: { "messages": [{"role":"user", "content": x["prompt"]+ "\n"+ x["chosen"][0]["content"]}]+ x["chosen"][1:] })

In [ ]:
columns_to_remove = [c for c in train_ds.column_names if c not in ["messages"]]
train_ds = train_ds.remove_columns(columns_to_remove)

columns_to_remove = [c for c in eval_ds.column_names if c not in ["messages"]]
eval_ds = eval_ds.remove_columns(columns_to_remove)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
model

In [ ]:
tokenizer.chat_template

In [ ]:
from helpers import set_padding_for_tokenizer
set_padding_for_tokenizer(tokenizer)

In [ ]:
from helpers import stream_responses_for_sample
from transformers import GenerationConfig

generation_config =  GenerationConfig(max_new_tokens=250, eos_token_id=[tokenizer.eos_token_id])
sample_conversations = [
    [{"role": "user", "content": "What is the capital of France?"}],
    [{"role": "user", "content": "Write me a javascript function that check if string is palindrome."}],
    [{"role": "user", "content": "Given x^2=36-4 what is x?"}]
]
stream_responses_for_sample(model, tokenizer, sample_conversations, generation_config=generation_config)

In [ ]:
print(model)

## Step 1: Enable DoRA

### **Purpose:**
Using Weight-Decomposition to improve LoRA stability and performance.

### **Line-by-Line Breakdown:**
- `use_dora=True`: Activates the DoRA specific logic in PEFT.

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                     "gate_proj", "up_proj","down_proj"],
    modules_to_save=["embed_tokens", "input_layernorm", "post_attention_layernorm"],
    bias="none",
    task_type="CAUSAL_LM",
    use_dora=True
)


In [ ]:
from peft import get_peft_model
peft_model = get_peft_model(model, peft_config)

In [ ]:
peft_model

In [ ]:
peft_model.base_model.model.model.layers[0].self_attn.q_proj.lora_magnitude_vector.training

In [ ]:
import os
from trl import SFTConfig, SFTTrainer

args = SFTConfig(
    output_dir=os.getenv("WANDB_RUN_ID"),
    report_to="wandb",
    num_train_epochs=1.0,
    do_train=True,
    do_eval=True,
    log_level="debug",
    gradient_checkpointing=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    per_device_eval_batch_size=1,
    lr_scheduler_type="constant",
    bf16=True,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=0.1,
    max_grad_norm=.3,
    learning_rate=1e-4
)


In [ ]:
trainer = SFTTrainer(
    model=peft_model,
    tokenizer=tokenizer,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds
)
trainer.train()